In [19]:
from __future__ import print_function, division
import cv2
import torch
import numpy as np
import pickle
import math

In [20]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model import sparse3DBA
from featureBA.src.utils import sobel_filter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
data = pickle.load(open("toy_example/data/toyexample_1_data.p", 'rb'))
img = cv2.imread("toy_example/data/toyexample_1.png")

In [22]:
T_perturbed = np.array([[math.cos(10*math.pi/180), -math.sin(10*math.pi/180), 0, 0],
             [math.sin(10*math.pi/180), math.cos(10*math.pi/180), 0, 0],
             [0, 0, 1, 0]])

In [23]:
data['coords'] = np.around(data['2d_points']).astype(int) - 1

In [24]:
P_perturbed = np.dot(data['K'], T_perturbed)
projected_2d = np.dot(P_perturbed, np.concatenate((data['3d_points'], np.ones(len(data['3d_points']))[:, None]),-1).T)
projected_2d = (projected_2d.T/projected_2d.T[:,2,None])[:, :2]

In [25]:
coords_2d = np.around(projected_2d)
coords_2d = coords_2d.astype(int) - 1

In [26]:
img = img.astype('uint8')
for i, p in enumerate(coords_2d):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)

0 [178 119]
1 [146 132]
2 [156  99]
3 [77 95]
4 [115 134]
5 [120 118]
6 [ 93 133]


In [27]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [141]:
cv2.imwrite('perturbed_points.png', img)

True

### run BA

In [10]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model2 import sparse3DBA
from featureBA.src.utils import (from_homogeneous, to_homogeneous,
                batched_eye_like, skew_symmetric, so3exp_map, sobel_filter, squared_loss, scaled_loss)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
img = cv2.imread("toy_example/data/toyexample_1.png", 0)
img = img.astype('double')

In [12]:
img_torch = torch.from_numpy(img)[None,...]
grad_x, grad_y = sobel_filter(img_torch)

In [13]:
img = grad_y.numpy().reshape(img.shape).astype('uint8')
for i, p in enumerate(coords_2d):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

0 [178 119]
1 [146 132]
2 [156  99]
3 [77 95]
4 [115 134]
5 [120 118]
6 [ 93 133]


In [120]:
cv2.imwrite("grad_x.png", grad_x.numpy().reshape(img.shape).astype('uint8'))
cv2.imwrite("grad_y.png", grad_y.numpy().reshape(img.shape).astype('uint8'))

True

In [14]:
pts3D = torch.from_numpy(data['3d_points'][:,:3])
ref2d = torch.from_numpy(np.flip(data['coords'], axis = 1).copy())
feature_ref = torch.cat([img_torch[:, i, j].unsqueeze(0) for i, j in zip(ref2d[:,0], ref2d[:,1])]).type(torch.DoubleTensor)
feature_map_query = img_torch.type(torch.DoubleTensor)
R_init, t_init = torch.from_numpy(T_perturbed[:, :3]), torch.from_numpy(T_perturbed[:, 3])
feature_grad_x = grad_x
feature_grad_y = grad_y
K = torch.from_numpy(data['K'])

In [17]:
model = sparse3DBA(n_iters = 100, lambda_ = 0)

In [18]:
R, t = model(pts3D, ref2d, feature_map_query, feature_grad_x, feature_grad_y, K, R_init, t_init)

prev cost is  tensor(23.3571, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([-0.0004, -0.0010,  0.0570], dtype=torch.float64)
dt is :  tensor([-0.0440,  1.2914, -0.1541], dtype=torch.float64)
new cost is  tensor(57., dtype=torch.float64)
cost increased, continue with next iteration
prev cost is  tensor(23.3571, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([-3.6683e-05, -1.0160e-04,  5.6957e-03], dtype=torch.float64)
dt is :  tensor([-0.0044,  0.1291, -0.0154], dtype=torch.float64)
new cost is  tensor(22.2857, dtype=torch.float64)
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([-2.7830e-05,  5.7225e-05,  5.8003e-03], dtype=torch.float64)
dt is :  tensor([-0.0234,  0.1379, -0.0159], dtype=torch.float64)
new cost is  tensor(25.0714, dtype=torch.float64)
cost increased, continue with next iteration

dr is :  tensor([ 5.4053e-08, -6.1849e-08,  3.3744e-07], dtype=torch.float64)
dt is :  tensor([-7.4031e-07,  8.3436e-06, -1.1462e-06], dtype=torch.float64)
new cost is  tensor(22.2857, dtype=torch.float64)
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([ 4.7766e-07, -5.4692e-07,  3.1473e-06], dtype=torch.float64)
dt is :  tensor([-4.8390e-06,  7.9862e-05, -1.0522e-05], dtype=torch.float64)
new cost is  tensor(22.9286, dtype=torch.float64)
cost increased, continue with next iteration
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([ 5.4053e-08, -6.1849e-08,  3.3744e-07], dtype=torch.float64)
dt is :  tensor([-7.4030e-07,  8.3436e-06, -1.1462e-06], dtype=torch.float64)
new cost is  tensor(22.2857, dtype=torch.float64)
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7,

dr is :  tensor([ 5.4850e-10, -6.2747e-10,  3.4009e-09], dtype=torch.float64)
dt is :  tensor([-7.7536e-09,  8.3827e-08, -1.1579e-08], dtype=torch.float64)
new cost is  tensor(22.9286, dtype=torch.float64)
cost increased, continue with next iteration
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([ 5.4850e-10, -6.2747e-10,  3.4009e-09], dtype=torch.float64)
dt is :  tensor([-7.7536e-09,  8.3827e-08, -1.1579e-08], dtype=torch.float64)
new cost is  tensor(22.9286, dtype=torch.float64)
cost increased, continue with next iteration
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([ 5.4850e-10, -6.2747e-10,  3.4009e-09], dtype=torch.float64)
dt is :  tensor([-7.7536e-09,  8.3827e-08, -1.1579e-08], dtype=torch.float64)
new cost is  tensor(22.9286, dtype=torch.float64)
cost increased, continue with next iteration
prev cost is  ten

J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([ 5.4850e-10, -6.2747e-10,  3.4009e-09], dtype=torch.float64)
dt is :  tensor([-7.7536e-09,  8.3827e-08, -1.1579e-08], dtype=torch.float64)
new cost is  tensor(22.9286, dtype=torch.float64)
cost increased, continue with next iteration
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([ 5.4850e-10, -6.2747e-10,  3.4009e-09], dtype=torch.float64)
dt is :  tensor([-7.7536e-09,  8.3827e-08, -1.1579e-08], dtype=torch.float64)
new cost is  tensor(22.9286, dtype=torch.float64)
cost increased, continue with next iteration
prev cost is  tensor(22.2857, dtype=torch.float64)
J_p_T is  torch.Size([7, 3, 6])
J_e_p is  torch.Size([7, 2, 3])
dr is :  tensor([ 5.4850e-10, -6.2747e-10,  3.4009e-09], dtype=torch.float64)
dt is :  tensor([-7.7536e-09,  8.3827e-08, -1.1579e-08], dtype=torch.float64)
new cost is  tensor(22.9286, dtype=torch.float64)
cost increased, continue with 

In [129]:
R

tensor([[ 9.9854e-01, -5.3988e-02,  4.6671e-04],
        [ 5.3990e-02,  9.9852e-01, -7.0458e-03],
        [-8.5632e-05,  7.0607e-03,  9.9998e-01]], dtype=torch.float64)

In [130]:
t

tensor([[ 0.9848, -0.1736,  0.0000],
        [ 0.1736,  0.9848,  0.0000],
        [ 0.0000,  0.0000,  1.0000]], dtype=torch.float64)

In [131]:
t

tensor([-2.2972, -0.6782,  0.0975], dtype=torch.float64)

In [349]:
a @ b

array([[[1.13206942, 1.11894092, 0.57274033, 1.24841041, 0.79335565,
         0.46561771],
        [1.2855546 , 1.19792658, 1.26153542, 1.5414339 , 1.17330447,
         1.04735982]]])